## nltk 영화 감상 corpus 기반 Word2Vec 사용법
### 출처: [데이터 사이언스 스쿨](https://datascienceschool.net/view-notebook/6927b0906f884a67b0da9310d3a581ee/)

### 단어 임베딩을 위한 Corpus 생성

In [1]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\EZEN\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
from nltk.corpus import movie_reviews
sentences = [list(s) for s in movie_reviews.sents()] #타입을 리스트로 바꿔주는 과정 아래보면 리스트처럼 생겼으나 타입이 리스트가 아님

In [4]:
movie_reviews.sents()

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.'], ...]

In [5]:
type(movie_reviews.sents())

nltk.corpus.reader.util.ConcatenatedCorpusView

In [6]:
len(sentences)

71532

In [7]:
sentences[10000]

['means', 'so', 'f', '*', '*', '*', 'ing', 'earnest', '.']

In [8]:
movie_reviews.sents()[10000]

['means', 'so', 'f', '*', '*', '*', 'ing', 'earnest', '.']

### 코퍼스를 입력 인수로 하여 Word2Vec 클래스 객체를 생성

In [9]:
from gensim.models.word2vec import Word2Vec

In [10]:
# 트레이닝 과정
%time model = Word2Vec(sentences)

Wall time: 3.84 s


In [11]:
# 트레이닝 종료후 메모리 반환(unload)
model.init_sims(replace=True)

In [12]:
model.wv.vectors.shape #14794차원을 100차원으로 줄였다.

(14794, 100)

In [13]:
dir(model.wv) # 이 객체의 속성을 알 수 있음. 

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_load_specials',
 '_log_evaluate_word_analogies',
 '_save_specials',
 '_smart_save',
 'accuracy',
 'add',
 'closer_than',
 'cosine_similarities',
 'distance',
 'distances',
 'doesnt_match',
 'evaluate_word_analogies',
 'evaluate_word_pairs',
 'get_keras_embedding',
 'get_vector',
 'index2entity',
 'index2word',
 'init_sims',
 'load',
 'load_word2vec_format',
 'log_accuracy',
 'log_evaluate_word_pairs',
 'most_similar',
 'most_similar_cosmul',
 'most_similar_to_given',
 'n_similarity',
 'rank',
 'relative_cosine_similarity',
 'save',
 'save_word2vec_for

### 사용가능한 메소드
#### - similarity : 두 단어의 유사도 계산

In [14]:
model.wv.similarity('actor', 'actress')

0.8767059

In [15]:
model.wv.similarity('he', 'she')

0.8608404

In [16]:
model.wv.similarity('actor', 'she')

0.22304484

#### - most_similar : 가장 유사한 단어를 출력

In [17]:
model.wv.most_similar("accident")

[('egyptian', 0.8689893484115601),
 ('boat', 0.8673871755599976),
 ('meeting', 0.8629583120346069),
 ('plane', 0.8579989075660706),
 ('affair', 0.8574287295341492),
 ('church', 0.8442820310592651),
 ('position', 0.8435735702514648),
 ('arena', 0.8435499668121338),
 ('abandoned', 0.8397265672683716),
 ('conversation', 0.8385642766952515)]

In [18]:
# she + (actor - actress)
model.wv.most_similar(positive=['she', 'actor'], negative='actress', topn=1)

[('he', 0.2972775101661682)]

## 네이버 영화 감상 코퍼스
raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

In [19]:
import codecs
#utf-8로 되어있는 것 아래처럼 읽어야함. 원래 자료형이 utf-8인코딩이여서 그럼... 파이썬은 원래 한글 읽음 euck인코딩
def read_data(filename):
    with codecs.open(filename, encoding='utf-8', mode='r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]   # header 제외
    return data

train_data = read_data('data/02_naver_ratings_train.txt')

In [20]:
len(train_data)

150000

In [21]:
print(train_data[:5]) #0은 부정 1은 긍정

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1']]


In [22]:
from konlpy.tag import Okt #오픈 코리안 텍스트 트위터로 되어있는거 이걸로 다 바꾸면 됌
tagger = Okt()

def tokenize(doc):
    return ['/'.join(t) for t in tagger.pos(doc, norm=True, stem=True)]

train_docs = [row[1] for row in train_data]

E:\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [23]:
# 시간이 매우 오래 걸림
%time sentences = [tokenize(d) for d in train_docs]

Wall time: 5min 10s


In [24]:
%time model = Word2Vec(sentences)
model.init_sims(replace=True)

Wall time: 6 s


In [33]:
model.wv.vectors.shape

(15409, 100)

In [25]:
sentences[10]

['걍/Adverb',
 '인피니트/Noun',
 '가/Josa',
 '짱/Noun',
 '이다/Josa',
 './Punctuation',
 '진짜/Noun',
 '짱/Suffix',
 '이다/Josa',
 '♥/Foreign']

In [26]:
model.wv.similarity(*tokenize(u'배우 여배우')) #utf-8 인코딩이라 앞에 u 붙임.

0.7199743

In [27]:
model.wv.similarity(*tokenize(u'배우 남자'))

0.30195475

In [28]:
# 남자 + (여배우 - 배우) = 여자
from konlpy.utils import pprint
pprint(model.wv.most_similar(positive=tokenize(
    u'남자 여배우'), negative=tokenize(u'배우'), topn=1))

[('여자/Noun', 0.8286821842193604)]


In [29]:
# 아빠 + (남자 - 여자) = 엄마
pprint(model.wv.most_similar(positive=tokenize(
    u'아빠 남자'), negative=tokenize(u'여자'), topn=1))

[('엄마/Noun', 0.878487765789032)]


In [37]:
pprint(model.wv.most_similar(positive=tokenize(
    u'발연기 배우'), negative=tokenize(u'연기자'), topn=1))

[('시나리오/Noun', 0.6567433476448059)]
